1

In [ ]:
#!/bin/python3
import math
import os
import random
import re
import sys
import datetime
import pandas as pd
import numpy as np
import random
random.seed(6666)

def random_data(financial_data):
    # You are going to create a new dataframe signals
    # you will implement here the logic
    
    x = financial_data["Close"]
    sig=[]
    for i in range(len(x)):
        if i%4 == 0:
            sig.append(0)
        else:
            sig.append(1)
    signals=pd.DataFrame(data={'signal':sig},index=financial_data.index)
    signals['orders']=float(0)
    signals['orders']=signals['signal'].diff()
    return signals

def calculate_holding_cash(financial_data,signals):
    # You are going to set your initial amount of money you want
    # to invest --- here it is 10,000
    cash = [10000]

    # You are going to create a new dataframe positions
    # Remember the index is still the same as signals
    stock = pd.DataFrame(index = signals.index)
    stock = np.where(signals['signal'] == 1, financial_data['Close'] * 10, 0)
    holding = stock.copy()
    portfolio = pd.DataFrame({'signal' : signals.signal, 'stock' : stock, 'order' : signals.orders, 'holdings' : holding})
    for index, rows in portfolio.iterrows():
        if index > 0:
            
            if rows['order'] == 1:
                cash = cash + [cash[index - 1] - financial_data.iloc[index]['Close'] *10]
            elif rows['order'] == -1:
                cash = cash + [cash[index - 1]+financial_data.iloc[index]['Close']*10]
        
            else:
                cash = cash +[cash[index - 1]]
        
    portfolio['stock'] = stock
    portfolio['cash'] = cash
    portfolio['total'] = portfolio['holdings'] + portfolio['cash']
    portfolio['returns'] = portfolio['total'].pct_change()
    # You are going to buy 10 shares of the stock when signal is 1
    # You are going to sell 10 shares of the stock when signal is -1
    # You will assign these values to the column stock of the
    # dataframe positions

    # You are now going to calculate the notional (quantity x price)
    # for your portfolio. You will multiply Close from
    # the dataframe containing prices and the positions (10 shares)
    # You will store it into the variable portfolio
   
    # Add `holdings` to portfolio
   
    # You will store positions.diff into pos_diff
   
    # You will now add a column cash in your dataframe portfolio
    # which will calculate the amount of cash you have
    # initial_capital - (the notional you use for your different buy/sell)
    
    # You will now add a column total to your portfolio calculating the part of holding
    # and the part of cash
    
    # Add `returns` to portfolio
    
    return portfolio[['stock','holdings','cash','total','returns']]


def test1(data):
    df=random_data(data)
    fptr.write(df.to_string())

def test2(data):
    df=random_data(data)
    portfolio=calculate_holding_cash(data,df)
    fptr.write(portfolio.to_string())
    

if __name__ == '__main__':
    #fptr = open(os.environ['OUTPUT_PATH'], 'w')
    fptr = sys.stdout
    tmp = input()
    row_num = int(input().strip())
    Data = []
    col_names = list(map(str, input().split('\t')))
    for i in range(row_num):

        line=list(map(str, input().split('\t')))
        line[0] = pd.to_datetime(line[0])
        line[1] = float(line[1])
        line[2] = int(line[2])
        Data.append(line)
    data = pd.DataFrame(Data, columns= col_names)
    data.set_index('Date')
    
    if tmp == '1':
      test1(data)
    elif tmp == '2':
      test2(data)
    else:
      raise RuntimeError('invalid input')


2

In [ ]:
#!/bin/python3

import math
import os
import random
import re
import sys
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#
# Complete the 'case1' function below.
#
# The function accepts STRING_ARRAY fp_data as parameter.
#

def case1(financial_data):
    # Print First 5 rows of the dataframe
    print(financial_data.head(5))
    # Print Last 5 rows of the dataframe
    print(financial_data.tail(5))
    # Print descriptive statistics of the dataframe
    print(financial_data.describe())
    pass

def case2(financial_data):
    # resample the dataframe to get the monthly mean prices
    # display the first 5 rows
    print(financial_data.resample('M').mean().head(5))
    pass

def case3(financial_data):
    # filter the dataframe down so that you are only working with the 'Adj Close' field.
    daily_close = financial_data[['Adj Close']]
    # print the first 5 rows
    #print (daily_close.head(5))
    # calculate the daily returns with respect to the 'Adj Close' field.
    returns = daily_close.pct_change()
    print (returns.dropna())
    # print the all the daily returns, and only the daily returns, as a dataframe, not a 
    # series object, being sure to exlcude any nan values.
    # NOTE: you should have a dataframe with one column, 'Adj Close', no nan values.
    



def case4(financial_data):
    # now calculate the cumulative daily returns with the output format being similar 
    # to case3 above.
    # print the results
    rets = financial_data[['Adj Close']].pct_change()
    cum_rets = (1+rets).cumprod().dropna()
    print (cum_rets)    
    pass

def case5(financial_data):
    # resample the daily returns to monthly mean of daily returns
    # print the results
    rets = financial_data[['Adj Close']].pct_change()
    cum_rets = (1+rets).cumprod()
    month_rets = cum_rets.resample('M').mean()
    print(month_rets)
    pass

def case6(financial_data):
    # isolate the adjusted closing prices and store it in a variable
    # calculate the moving average for a window of 20. That is the mean of the 
    # adjusted closes over the last 20 days
    # print the last 20 daily moving averages

    daily_close = financial_data[['Adj Close']]
    ma = daily_close.rolling(20).mean()
    print(ma)
    

def case7(financial_data):
    # Calculate the daily volatility over a 100-day window.
    # don't forget to multiply by square root
    # don't forget that you need to use pct_change
    daily_close = financial_data[['Adj Close']].pct_change()
    ma = daily_close.rolling(100)
    vol = ma.std()* np.sqrt(100)
    print(vol)
    pass


def case8(financial_data):
    # Initialize the short rolling window (window=50)
    # Initialize the long rolling window (window=100)
    daily_close = financial_data[['Adj Close']].pct_change()
    short_wind = 50
    long_wind = 100
    # You will create a signal dataframe using the index of financial_data
    signals = pd.DataFrame(index = financial_data.index)
    # You will assign 0 to the column signal of the dataframe signals
    financial_data['signal'] = 0
    # Create short simple moving average over the short window
    # Create long simple moving average over the long window
    close_data = financial_data.copy().drop(['High','Low','Open','Adj Close','Volume'],axis=1)
    short_ma = close_data.rolling(short_wind, min_periods = 1).mean()
    short_ma1 = short_ma['Close'] 
    long_ma = close_data.rolling(long_wind, min_periods =1).mean()
    long_ma1 = long_ma['Close'] 
    # You will now conditionally fill the signal field.
    # When the small window moving average is higher than the long window 
    # the signal will be 1 else 0.
    if len(short_ma1) > len(long_ma1):
        financial_data['signal'] = 1.0
    else:
        financial_data['signal'] = 0.0
    
    # Generate trading orders by inserting in a new column orders
    # 1 if it is a buy order -1 if it is a sell order
    # you should just use the diff command on the column signal
    #financial_data['trading_orders'] = []
    #if financial_data['signal'] == 1:
    #    financial_data['trading_orders'] = 1
    #if financial_data['signals'] == 0:
    #    financial_data['trading_orders'] =-1
        
    ma_diff = short_ma1 - long_ma1
    signal_list = np.where(ma_diff > 0, 1.0,0.0)
    signals['signal'] = signal_list
    signals['short_mavg'] = short_ma1
    signals['long_mavg'] = long_ma1
    signals['orders'] = signals['signal'].diff()
    # Print the dataframe signals
    print (signals)

    pass

def case9(financial_data):
    # You will need to use the dataframe signals
    signal =  [0,1]
    signals = pd.DataFrame({'signal':signal})
    signals['orders'] = signals.signal.diff()
    
    # You will now conditionally fill the signal field.
    # When the small window moving average is higher than the long window 
    # the signal will be 1 else 0.


    # You are going to set your initial amount of money you want
    # to invest --- here it is 10,000
    initial_capital = 10000
    
    # You are going to create a new dataframe positions
    # Remember the index is still the same as signals
    portfolio = pd.DataFrame(index = signals.index)
    
    # You are going to buy 10 shares of MSFT when signal is 1
    # You are going to sell 10 shares of MSFT when signal is -1
    # You will assign these values to the column MSFT of the
    # dataframe positions
    positions = pd.DataFrame(index = signals.index)
    positions['MSFT'] = 0
    for index, rows in positions.iterrows():
        holdings = 0
        if signals['orders'] == 1:
            holdings += financial_data[['Adj Close']]* 10
        elif signals['orders'] == -1:
            holdings -= financial_data[['Adj Close']]* 10
        else:
            holdings = holdings
        return holdings
        

        
  
    # You are now going to calculate the notional (quantity x price)
    # for your portfolio. You will multiply Adj Close from
    # the dataframe containing prices and the positions (10 shares)
    # You will store it into the variable portfolio

    # Add `holdings` to portfolio
    portfolio['holdings'] = holdings
    
    # You will store positions.diff into pos_diff
    pos_diff = positions.diff()
    # You will now add a column cash in your dataframe portfolio
    # which will calculate the amount of cash you have
    # initial_capital - (the notional you use for your different buy/sell)
    cash = initial_capital
    
    #for index, rows in positions.iterrows():
        #if financial_data['signal'] == 1:
            #cash += financial_data[['Adj Close']]* positions
        #if financial_data['signal'] == -1:
            #cash -= financial_data[['Adj Close']]* positions
    #portfolio['cash'] = cash
    
    portfolio['cash'] = initial_capital - ((pos_diff.multiply(financial_data['Adj Close'], axis=0)).sum(axis=1).cumsum())
    
    # You will now add a column total to your portfolio calculating the part of holding
    # and the part of cash
    portfolio['total'] = portfolio['cash']+ portfolio['holdings']
    
    # Add `returns` to portfolio
    portfolio['return'] = portfolio['total'].pct_change()
    
    # Print the first lines of `portfolio`
    print (portfolio.head())
    
    pass
if __name__ == '__main__':
    case_number=input().strip()
    df = pd.read_csv(sys.stdin, header=0, index_col='Date', parse_dates=True)
    globals()['case'+case_number](df)


### 3

In [ ]:
# Python program to get average of a list
def average(lst):
    return sum(lst) / len(lst)

class TradingStrategy:
    def __init__(self):
        self.small_window=deque()
        self.large_window=deque()
        self.long_signal=False
        self.position=0
        self.cash=10000
        self.total=0
        self.holdings=0

    def onPriceUpdate(self,price_update):
    
        # you should store 50 close prices for the small window
        self.small_window.append(price_update["close"])
        if len(self.small_window) > 49:
            self.small_window.popleft()
        # you should store 100 close prices for the large window
        self.large_window.append(price_update["close"])
        if len(self.large_window) > 99:
            self.large_window.popleft()
        # when the number of close prices is higher than 50, you need to discard the older one (small window)
        # when the number of close prices is higher than 100, you need to discard the older one (large window)
        # you compare the small window vs the large window
        s = self.small_window
        l = self.large_window
        self.ave_small = average(s)
        self.ave_large = average(l)
        if self.ave_small > self.ave_large:
            self.long_signal = True
        else:
            self.long_signal = False
        #if small window > large window generate long signal
        #if not long signal = false
        # Call the function checkSignal
        self.checkSignal(price_update)

    def checkSignal(self,price_update):
        #for i in range(len(price_update)):
        if self.long_signal == True and self.position == 0:
            print(price_update["date"] + " send buy order for 10 shares price=" + str(price_update['adjprice']))
            self.cash = self.cash - 10 * price_update["adjprice"]
            self.position = self.position + 10 
            self.holdings = int(10 * price_update["adjprice"])
            self.total = self.cash + self.holdings
        # if there is a long signal and the position is 0 
        # you should send an order
        # for that you will use print, you will display the timestamp and write that you send a buy order
        #     example:
        #     print(price_update['date'] +
        #          " send buy order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
        if self.position > 0 and self.long_signal == False:
            print(price_update['date']+" send sell order for 10 shares price=" + str(price_update['adjprice']))
            self.cash = self.cash + 10 * price_update["adjprice"]
            self.position = self.position - 10
            self.holdings = int(self.position * price_update["adjprice"])
            self.total = self.cash + self.holdings
        # Now if the position is positive and there is no long signal any more
        # You need to send a sell order
        #    print(price_update['date']+
        #          " send sell order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
    
        # For each iteration, you need to manage holdings
        # For each iteration, you need to know how much in total you have (total=holdings + cash)
        # You need to display the following for each iteration
        print('%s total=%d, holding=%d, cash=%d' %
            (price_update['date'],self.total, self.holdings, self.cash))

In [ ]:
#!/bin/python3

import math
import os
import random
import re
import sys

from collections import deque


#
# Complete the 'TradingStrategy' class below.
#
#



# Python program to get average of a list
def average(lst):
    return sum(lst) / len(lst)

class TradingStrategy:
    def __init__(self):
        self.small_window=deque()
        self.large_window=deque()
        self.long_signal=False
        self.position=0
        self.cash=10000
        self.total=0
        self.holdings=0

    def onPriceUpdate(self,price_update):
        # you should store 50 close prices for the small window
        # you should store 100 close prices for the large window
        # when the number of close prices is higher than 50, you need to discard the older one (small window)
        # when the number of close prices is higher than 100, you need to discard the older one (large window)
        # you compare the small window vs the large window
        # if small window > large window generate long signal
        # if not long signal = false
        # Call the function checkSignal
        self.small_window.append(price_update['close'])
        if len(self.small_window) > 50:
            self.small_window.popleft()
            
        
        self.large_window.append(price_update['close'])
        if len(self.large_window) > 100:
            self.small_window.popleft()
            
        if len(self.small_window) == 50:
            if self.long_signal == True:
                if average(self.small_window) > average(self.large_window):
                    self.long_signal = True
                    self.position = 0
                else:
                    self.long_signal = False
                    self.position = -1
                    
            if self.long_signal == False:
                if average(self.small_window) > average(self.large_window):
                    self.long_signal = True
                    self.position = 1
                else:
                    self.long_signal = False
                    self.position = 0
            
            self.checkSignal(price_update)

    def checkSignal(self,price_update):
        # if there is a long signal and the position is 0 
        # you should send an order
        # for that you will use print, you will display the timestamp and write that you send a buy order
        #     example:
        #     print(price_update['date'] +
        #          " send buy order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
        if self.position == 1:
            print(price_update['date']+ " send buy order for 10 shares price=" + str(price_update['adjprice']))
            self.cash -= price_update['adjprice'] * 10
            
        # Now if the position is positive and there is no long signal any more
        # You need to send a sell order
        #    print(price_update['date']+
        #          " send sell order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
        if self.position == -1:
            print(price_update['date']+ " send buy order for 10 shares price=" + str(price_update['adjprice']))
            self.cash += price_update['adjprice'] * 10
        # For each iteration, you need to manage holdings
        if self.long_signal == True:
            self.holdings = price_update['adjprice'] * 10
        else:
            self.holdings = 0
        # For each iteration, you need to know how much in total you have (total=holdings + cash)
        self.total = self.cash + self.holdings 
        # You need to display the following for each iteration
        print('%s total=%d, holding=%d, cash=%d' %
              (price_update['date'],self.total, self.holdings, self.cash))

if __name__ == '__main__':
    
    ts=TradingStrategy()
    nb_of_rows = int(input().strip())
    market_data_header = input().strip()

    for _ in range(nb_of_rows):
        row = input().strip().split(',')
        ts.onPriceUpdate({'date' : row[0],
                              'close' : float(row[4]),
                              'adjprice' : float(row[6])})
